In [7]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
from datetime import datetime

In [9]:
pd.set_option('display.max_columns', None)

In [5]:
file_SIM = '../Data/Sistema de Informação sobre Mortalidade – SIM/DO23OPEN.csv'

In [16]:
cid10 = '../Data/cid10/CID-10-SUBCATEGORIAS.CSV'

In [8]:
data_SIM = pd.read_csv(file_SIM, sep=';', low_memory=False)


In [10]:
for col in data_SIM.columns:
    tipos = data_SIM[col].apply(type).nunique()
    if tipos > 1:
        print(f"⚠️ Coluna '{col}' tem múltiplos tipos de dados. Convertendo para string.")
        data_SIM[col] = data_SIM[col].astype(str)


⚠️ Coluna 'LINHAA' tem múltiplos tipos de dados. Convertendo para string.
⚠️ Coluna 'LINHAB' tem múltiplos tipos de dados. Convertendo para string.
⚠️ Coluna 'LINHAC' tem múltiplos tipos de dados. Convertendo para string.
⚠️ Coluna 'LINHAD' tem múltiplos tipos de dados. Convertendo para string.
⚠️ Coluna 'LINHAII' tem múltiplos tipos de dados. Convertendo para string.
⚠️ Coluna 'STCODIFICA' tem múltiplos tipos de dados. Convertendo para string.
⚠️ Coluna 'CODIFICADO' tem múltiplos tipos de dados. Convertendo para string.
⚠️ Coluna 'VERSAOSIST' tem múltiplos tipos de dados. Convertendo para string.
⚠️ Coluna 'CAUSAMAT' tem múltiplos tipos de dados. Convertendo para string.
⚠️ Coluna 'FONTES' tem múltiplos tipos de dados. Convertendo para string.
⚠️ Coluna 'TPNIVELINV' tem múltiplos tipos de dados. Convertendo para string.
⚠️ Coluna 'CAUSABAS_O' tem múltiplos tipos de dados. Convertendo para string.
⚠️ Coluna 'TPPOS' tem múltiplos tipos de dados. Convertendo para string.
⚠️ Coluna 'CB_AL

In [11]:
data_SIM.head()

,contador,ORIGEM,TIPOBITO,DTOBITO,HORAOBITO,NATURAL,CODMUNNATU,DTNASC,IDADE,SEXO,RACACOR,ESTCIV,ESC,ESC2010,SERIESCFAL,OCUP,CODMUNRES,LOCOCOR,CODESTAB,CODMUNOCOR,IDADEMAE,ESCMAE,ESCMAE2010,SERIESCMAE,OCUPMAE,QTDFILVIVO,QTDFILMORT,GRAVIDEZ,SEMAGESTAC,GESTACAO,PARTO,OBITOPARTO,PESO,TPMORTEOCO,OBITOGRAV,OBITOPUERP,ASSISTMED,EXAME,CIRURGIA,NECROPSIA,LINHAA,LINHAB,LINHAC,LINHAD,LINHAII,CAUSABAS,CB_PRE,COMUNSVOIM,DTATESTADO,CIRCOBITO,ACIDTRAB,FONTE,NUMEROLOTE,DTINVESTIG,DTCADASTRO,ATESTANTE,STCODIFICA,CODIFICADO,VERSAOSIST,VERSAOSCB,FONTEINV,DTRECEBIM,ATESTADO,DTRECORIGA,OPOR_DO,CAUSAMAT,ESCMAEAGR1,ESCFALAGR1,STDOEPIDEM,STDONOVA,DIFDATA,NUDIASOBCO,DTCADINV,TPOBITOCOR,DTCONINV,FONTES,TPRESGINFO,TPNIVELINV,DTCADINF,MORTEPARTO,DTCONCASO,ALTCAUSA,CAUSABAS_O,TPPOS,TP_ALTERA,CB_ALT
0,1,1,2,14022023,930.0,833.0,330270.0,10071954.0,468,1,4.0,1.0,2.0,1.0,NaN,715210.0,330190,3,NaN,330190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,2.0,*R99X,*I10X,nan,nan,nan,I10,NaN,NaN,14022023.0,NaN,NaN,NaN,20230018.0,NaN,23022023.0,5.0,S,S,3.2.30,3.4,NaN,27022023.0,R99/I10,27022023,13,nan,NaN,10.0,0.0,1,13,NaN,NaN,NaN,NaN,nan,NaN,nan,NaN,NaN,NaN,NaN,I10,nan,NaN,nan
1,2,1,2,14022023,1730.0,822.0,221110.0,12091956.0,466,1,4.0,2.0,4.0,3.0,NaN,21210.0,221100,1,2323281.0,221100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,2.0,*R578,*I619,*I678,nan,nan,I678,NaN,NaN,14022023.0,NaN,NaN,NaN,20230028.0,NaN,17022023.0,5.0,S,S,3.2.30,3.4,NaN,17022023.0,R578/I619/I678,17022023,3,nan,NaN,12.0,0.0,1,3,NaN,NaN,NaN,NaN,nan,NaN,nan,NaN,NaN,NaN,NaN,I678,nan,NaN,nan
2,3,1,2,14022023,500.0,822.0,220790.0,10011953.0,470,2,NaN,3.0,9.0,9.0,NaN,999992.0,220790,1,2726971.0,221100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,2.0,*A419,*K750,nan,nan,*E149*I10X,K750,NaN,NaN,14022023.0,NaN,NaN,NaN,20230028.0,NaN,17022023.0,5.0,S,S,3.2.30,3.4,NaN,17022023.0,A419/K750*E149 I10,17022023,3,nan,NaN,9.0,0.0,1,3,NaN,NaN,NaN,NaN,nan,NaN,nan,NaN,NaN,NaN,NaN,K750,nan,NaN,nan
3,4,1,2,14022023,2235.0,823.0,231180.0,15121943.0,479,2,1.0,2.0,4.0,3.0,NaN,763010.0,230440,1,2497654.0,230440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,2.0,*I611,nan,nan,nan,*J189,I611,NaN,NaN,15022023.0,NaN,NaN,NaN,20230007.0,23022023.0,15022023.0,1.0,S,S,3.2.30,NaN,3.0,9032023.0,I611*J189,2032023,16,nan,NaN,12.0,0.0,1,23,NaN,NaN,NaN,NaN,nan,NaN,nan,NaN,NaN,NaN,NaN,I64,S,NaN,nan
4,5,1,2,14022023,940.0,841.0,411350.0,9041973.0,449,2,4.0,1.0,1.0,0.0,NaN,999992.0,510515,2,7050577.0,510515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,2.0,3.0,1.0,NaN,NaN,2.0,*J960,*C189,nan,nan,nan,C189,NaN,NaN,14022023.0,NaN,NaN,NaN,20240057.0,14022023.0,14022023.0,1.0,S,S,3.2.30,3.4,3.0,30102024.0,J960/C189,16022023,2,nan,NaN,0.0,0.0,1,624,3.0,17022023.0,9.0,17022023.0,nan,NaN,M,NaN,NaN,NaN,NaN,C189,S,NaN,nan


In [12]:
data_SIM.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465610 entries, 0 to 1465609
Data columns (total 86 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   contador    1465610 non-null  int64  
 1   ORIGEM      1465610 non-null  int64  
 2   TIPOBITO    1465610 non-null  int64  
 3   DTOBITO     1465610 non-null  int64  
 4   HORAOBITO   1416392 non-null  float64
 5   NATURAL     1399908 non-null  float64
 6   CODMUNNATU  1381624 non-null  float64
 7   DTNASC      1462816 non-null  float64
 8   IDADE       1465610 non-null  int64  
 9   SEXO        1465610 non-null  int64  
 10  RACACOR     1442526 non-null  float64
 11  ESTCIV      1399251 non-null  float64
 12  ESC         1375905 non-null  float64
 13  ESC2010     1367238 non-null  float64
 14  SERIESCFAL  404622 non-null   float64
 15  OCUP        1263542 non-null  float64
 16  CODMUNRES   1465610 non-null  int64  
 17  LOCOCOR     1465610 non-null  int64  
 18  CODESTAB    1064156 no

In [13]:
causas = data_SIM[['CAUSABAS']].drop_duplicates()
causas

,CAUSABAS
0,I10
1,I678
2,K750
3,I611
4,C189
...,...
1456528,V142
1458926,D033
1459361,A239
1461469,E611


## Carga dos dados

### Conexão com banco

In [ ]:
DB_USER = "postgres"
DB_PASSWORD = "root"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "mortalidades"

In [ ]:
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

In [ ]:
with engine.connect() as conn:
    print("✅ Conectado ao banco mortalidade")

In [ ]:
def inserir_dados(df, tabela):
    df.to_sql(tabela, con=engine, if_exists="append", index=False)
    print(f"✅ Dados inseridos na tabela {tabela}")

### Inserção dos dados

In [14]:
causas.rename(columns={'CAUSABAS': 'codigo_cid'}, inplace=True)

In [15]:
# Adicionar uma coluna vazia para a descrição
causas['descricao'] = "DESCONHECIDO"

In [ ]:
causas.to_sql("causas", con=engine, if_exists="append", index=False)

print("✅ Dados de CIDs inseridos com sucesso na tabela `causas`!")

In [17]:
df_cid10 = pd.read_csv(cid10, sep=';', encoding='latin1')

In [18]:
df_cid10

,SUBCAT,CLASSIF,RESTRSEXO,CAUSAOBITO,DESCRICAO,DESCRABREV,REFER,EXCLUIDOS,Unnamed: 8
0,A000,NaN,NaN,NaN,"Cólera devida a Vibrio cholerae 01, biótipo ch...",A00.0 Colera dev Vibrio cholerae 01 biot cholerae,NaN,NaN,NaN
1,A001,NaN,NaN,NaN,"Cólera devida a Vibrio cholerae 01, biótipo El...",A00.1 Colera dev Vibrio cholerae 01 biot El Tor,NaN,NaN,NaN
2,A009,NaN,NaN,NaN,Cólera não especificada,A00.9 Colera NE,NaN,NaN,NaN
3,A010,NaN,NaN,NaN,Febre tifóide,A01.0 Febre tifoide,NaN,NaN,NaN
4,A011,NaN,NaN,NaN,Febre paratifóide A,A01.1 Febre paratifoide A,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
12446,U818,NaN,NaN,NaN,Agente resistente a outros antibióticos relaci...,U81.8 Agente resist outr antibiót relac vancom...,NaN,NaN,NaN
12447,U88,NaN,NaN,NaN,Agente resistente a múltiplos antibióticos,U88 Agente resistente a múltiplos antibióticos,NaN,NaN,NaN
12448,U898,NaN,NaN,NaN,Agente resistente a outro antibiótico especifi...,U89.8 Agente resist outro antibiótico espec único,NaN,NaN,NaN
12449,U899,NaN,NaN,NaN,Agente resistente a antibiótico não especificado,U89.9 Agente resistente a antibiótico NE,NaN,NaN,NaN


In [19]:
df_cid10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12451 entries, 0 to 12450
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   SUBCAT      12451 non-null  object 
 1   CLASSIF     378 non-null    object 
 2   RESTRSEXO   874 non-null    object 
 3   CAUSAOBITO  1291 non-null   object 
 4   DESCRICAO   12451 non-null  object 
 5   DESCRABREV  12451 non-null  object 
 6   REFER       157 non-null    object 
 7   EXCLUIDOS   16 non-null     object 
 8   Unnamed: 8  0 non-null      float64
dtypes: float64(1), object(8)
memory usage: 875.6+ KB


In [ ]:
query = "SELECT id_causa, codigo_cid, descricao FROM causas WHERE descricao = 'DESCONHECIDO'"
df_causas = pd.read_sql(query, con=engine)

In [ ]:
causas = causas.merge(df_cid10[['SUBCAT', 'DESCRICAO']], 
                            left_on="codigo_cid", 
                            right_on="SUBCAT", 
                            how="left")

In [ ]:
causas.drop(columns=['SUBCAT'], inplace=True)

In [ ]:
causas.info()

In [ ]:
num_updates = (causas['descricao'] != causas['DESCRICAO']).sum()
if num_updates == 0:
    print("✅ As descrições já estão corretas! Nenhuma atualização necessária.")
else:
    with engine.connect() as conn:
        for _, row in causas.iterrows():
            query = text("UPDATE causas SET descricao = :descricao WHERE codigo_cid = :codigo_cid")
            conn.execute(query, {"descricao": row['DESCRICAO'], "codigo_cid": row['codigo_cid']})
        conn.commit()

    print(f"✅ Atualização concluída! {num_updates} registros foram modificados.")
